In [1]:
import numpy as np

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points

Loading BokehJS ...

In [8]:
# Set up constants and parameters in SI units
m = 9.109e-31; #Define electron mass
hbar = 6.636e-34/2/np.pi; #Reduced planck constant
size = 100e-9; #Define the width of the potential well in real space
Emax = 2.0; # Pick the highest energy (eV) the electron could have by distribution
k0 = np.pi/size; #Lowest allowed k in reciprocal space

# Set up the variable space
N = int(np.sqrt((Emax*1.602e-19*2*m)/hbar**2)/k0); # calculate the number of allowed states by the energy constraint
k = np.linspace(-int(np.sqrt((Emax*1.602e-19*2*m)/hbar**2))/1e9, int(np.sqrt((Emax*1.602e-19*2*m)/hbar**2))/1e9, 2*N); # Double the variable space by symmetry
En = 6.242e+18*(hbar*k*1e9)**2/(2*m);  #Energy for a free electron in eV

In [9]:
# Plot the dispersion relation of the free electrons
freeE = figure(plot_height=350, plot_width=300, title="Dispersion relationship of a free electron",
              tools="pan,reset,save,wheel_zoom", x_range = [np.amin(k), np.amax(k)],
              y_range = [0,1.1*np.amax(En)]);
freeE.xaxis[0].axis_label='k (nm-1)';
freeE.yaxis[0].axis_label='Energy (eV)';
source = ColumnDataSource(data = {'xVal':k, 'yVal':En});
freeE.circle('xVal', 'yVal', source = source, line_width=1, line_alpha=0.2);

In [10]:
# Plot the histogram of states per energy interval
histoN = 100; # Choose the number of bars in the histogram
bWidth = np.amax(En)/histoN; # Define the bin width of the histogram
kCount = np.empty(histoN, dtype = np.int8); # Declare an array to store histogram information
Ens = range(histoN)*bWidth;
zero = np.zeros(histoN);
for idx in range(histoN):
    kCount[idx] = len(En[ (En > idx*bWidth) & (En <= (idx+1)*bWidth) ]);
histo = figure(plot_height=350, plot_width=300, title="allowed states per Energy interval",
              tools="pan,reset,save,wheel_zoom", y_range = [0, np.amax(En)],
              x_range = [0, 1.1*np.amax(kCount)]);
histo.xaxis[0].axis_label='Number of states';
histo.yaxis[0].axis_label='Energy (eV)';
source1 = ColumnDataSource(data = {'bottom': Ens, 'top': Ens + bWidth, 'right': kCount, 'left': zero});
histo.quad(bottom = 'bottom', top = 'top', right = 'right', left = 'left', source = source1,
           fill_color="navy", line_color="white", alpha=1);

In [11]:
#calculate Density of states
gE = 1/(np.pi*hbar)*np.sqrt(m/(2*1.602e-19*En));
gEplot = figure(plot_height=350, plot_width=300, title="Density of States (1D)",
              tools="pan,reset,save,wheel_zoom", x_range=[np.amin(En), Emax], y_range=[0.5*np.amin(gE), 5*np.amin(gE)])
gEplot.xaxis[0].axis_label='Energy (eV)';
gEplot.yaxis[0].axis_label='Density of States';
source2 = ColumnDataSource(data = {'xVal': En, 'yVal': gE});
gEplot.line('xVal', 'yVal', source = source2, line_width=3, line_alpha=0.6)

bokeh.models.renderers.GlyphRenderer(
    id='21483733-e702-42f3-b030-84ebdcfa806d',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='5907bc0f-18ea-4b27-819d-a2908845835e',
        callback=None,
        data={'xVal': {'__ndarray__': 'iSDEur3//z8kJ7xJgrj/P3fyLHqWcf8/hYIWTPoq/z9L13i/reT+P8zwU9Swnv4/Bc+nigNZ/j/4cXTipRP+P6HZuduXzv0/BAZ4dtmJ/T8e966yakX9P/WsXpBLAf0/gSeHD3y9/D/IZigw/Hn8P8hqQvLLNvw/gDPVVevz+z/zwOBaWrH7PxsTZQEZb/s//yliSSct+z+bBdgyhev6P/Clxr0yqvo//gou6i9p+j/GNA64fCj6P0QjZycZ6Pk/fdY4OAWo+T9uToPqQGj5PxmLRj7MKPk/e4yCM6fp+D+YUjfK0ar4P2vdZAJMbPg/+ywL3BUu+D9AQSpXL/D3P0AawnOYsvc/+LfSMVF19z9pGlyRWTj3P5RBXpKx+/Y/dy3ZNFm/9j8U3sx4UIP2P2ZTOV6XR/Y/dY0e5S0M9j88jHwNFNH1P7xPU9dJlvU/8teiQs9b9T/nJGtPpCH1P5A2rP3I5/Q/8wxmTT2u9D8OqJg+AXX0P+UHRNEUPPQ/cSxoBXgD9D+5FQXbKsvzP7rDGlItk/M/czapan9b8z/jbbAkISTzPw1qMIAS7fI/8CopfVO28j+NsJob5H/yP+D6hFvESfI/7wnoPPQT8j+33cO/c97xPzd2GORCqfE/btPlqWF08T9h9SsR0D/xPwrc6hmOC/E/bIcixJvX8D+J99IP+aPwP18s/PylcPA/7SWei6I98D8z5Li77grwP2bOmBoVse8/112xAOxM7z+6drspYunuPwwZt5V3hu4/00SkRCwk7j8M+oI2gMLtP7g4U2tzYe0/0wAV4wUB7T9jUsidN6HsP14tbZsIQuw/05ED3Hjj6z+yf4tfiIXrPwn3BCY3KOs/0PdvL4XL6j8Kgsx7cm/qP7OVGgv/E+o/zzJa3Sq56T9bWYvy9V7pP1sJrkpgBek/zELC5Wms6D+yBcjDElToPwVSv+Ra/Oc/zCeoSEKl5z8Dh4LvyE7nP6xvTtnu+OY/yeELBrSj5j9U3bp1GE/mP1RiWygc++U/x3DtHb+n5T+rCHFWAVXlP/8p5tHiAuU/xtRMkGOx5D/9CKWRg2DkP6jG7tVCEOQ/wg0qXaHA4z9R3lYnn3HjP1E4dTQ8I+M/wRuFhHjV4j+liIYXVIjiP/l+ee3OO+I/wf5dBunv4T/5BzRioqThP6Sa+wD7WeE/vra04vIP4T9LXF8HisbgP0yL+27AfeA/u0OJGZY14D8+CxEOFtzfP+mh8m4+Tt8/dUu3VaXB3j/mB1/CSjbePzvX6bQurN0/dLlXLVEj3T+PrqgrspvcP4223K9RFdw/b9HzuS+Q2z84/+1JTAzbP+E/y1+nido/b5OL+0AI2j/g+S4dGYjZPzRztcQvCdk/av8e8oSL2D+GnmulGA/YP4dQm97qk9c/aRWunfsZ1z8v7aPiSqHWP9jXfK3YKdY/Z9U4/qSz1T/X5dfUrz7VPywJWjH5ytQ/ZD+/E4FY1D+AiAd8R+fTP4DkMmpMd9M/YVNB3o8I0z8q1TLYEZvSP9NpB1jSLtI/YRG/XdHD0T/Ry1npDlrRPyeZ1/qK8dA/YHk4kkWK0D97bHyvPiTQP/LkRqXsfs8/vBZb99i3zj9HbjVVQvPNP5vr1b4oMc0/tI48NIxxzD+aV2m1bLTLP0JGXELK+co/sFoV26RByj/plJR//IvJP+j02S/R2Mg/rXrl6yIoyD85Jrez8XnHP4v3Toc9zsY/pe6sZgYlxj+IC9FRTH7FPzNOu0gP2sQ/o7ZrS084xD/ZROJZDJnDP9r4HnRG/MI/n9Ihmv1hwj8t0urLMcrBP4H3eQnjNME/nULPUhGiwD9+s+qnvBHAP1OUmBHKB78/Mg3o6hTxvT+j0cPbWd+8P6DhK+SY0rs/Lz0gBNLKuj9J5KA7Bci5P/PWrYoyyrg/KhVH8VnRtz/wnmxve922P0V0HgWX7rU/JpVcsqwEtT+XASd3vB+0P5W5fVPGP7M/Ib1gR8pksj88DNBSyI6xP+amy3XAvbA/ORqnYGXjrz/Hfc8EPlWuP294ENgK0aw/NApq2stWqz8TM9wLgeapPxLzZmwqgKg/LEoK/Mcjpz9kOMa6WdGlP7W9mqjfiKQ/J9qHxVlKoz+zjY0RyBWiP1nYq4wq66A/QHTFbQKVnz8EZmQgmGedPwCGNDEWTps/M9Q1oHxImT+hUGhty1aXP0z7y5gCeZU/LNRgIiKvkz9E2yYKKvmRP5cQHlAaV5A/ROiM6OWRjT/LC0DtZ52KP8aLVa660Ic/M2jNK94rhT8Soadl0q6CP2I25FuXWYA/SlAGHVpYfD+z7Aj7Jk14PwBC0FGVkXQ/NFBcIaUlcT+RLlrTrBJsP4QuhVVSeWY/P6A5yTp/YT+EB+9czEhaP+OxfQqp0VI/J4A+NhcyST9Ix0470Hs+PxI52CwTGy8/IQBU92llFj8AOS4xXujjPgA5LjFe6OM+IQBU92llFj8SOdgsExsvP0jHTjvQez4/J4A+NhcyST/jsX0KqdFSP0IH71zMSFo/GKA5yTp/YT9ZLoVVUnlmP2IuWtOsEmw/GVBcIaUlcT/mQdBRlZF0P5XsCPsmTXg/KVAGHVpYfD9QNuRbl1mAP/+gp2XSroI/H2jNK94rhT+vi1WuutCHP7QLQO1nnYo/KuiM6OWRjT+JEB5QGleQPzbbJgoq+ZE/G9RgIiKvkz87+8uYAnmVP5JQaG3LVpc/JdQ1oHxImT/whTQxFk6bP/JlZCCYZ50/LnTFbQKVnz9R2KuMKuugP6iNjRHIFaI/HNqHxVlKoz+svZqo34ikP1c4xrpZ0aU/IUoK/Mcjpz8I82ZsKoCoPwsz3AuB5qk/KApq2stWqz9keBDYCtGsP7p9zwQ+Va4/MBqnYGXjrz/fpst1wL2wPzUM0FLIjrE/G71gR8pksj+MuX1Txj+zP5ABJ3e8H7Q/HZVcsqwEtT88dB4Fl+61P+eebG973bY/JBVH8VnRtz/r1q2KMsq4P0PkoDsFyLk/KD0gBNLKuj+b4SvkmNK7P5rRw9tZ37w/Kw3o6hTxvT9KlJgRyge/P3uz6qe8EcA/mULPUhGiwD9993kJ4zTBPynS6ssxysE/m9Ihmv1hwj/U+B50RvzCP9VE4lkMmcM/n7ZrS084xD8tTrtID9rEP4IL0VFMfsU/oO6sZgYlxj+G906HPc7GPzEmt7Pxecc/o3rl6yIoyD/d9Nkv0djIP96UlH/8i8k/p1oV26RByj80RlxCyvnKP4xXabVstMs/qo48NIxxzD+N69W+KDHNPzduNVVC880/rBZb99i3zj/m5Eal7H7PP3JsfK8+JNA/V3k4kkWK0D8fmdf6ivHQP8zLWekOWtE/WhG/XdHD0T/MaQdY0i7SPyTVMtgRm9I/XVNB3o8I0z955DJqTHfTP3uIB3xH59M/ZD+/E4FY1D8sCVox+crUP9fl19SvPtU/Z9U4/qSz1T/Y13yt2CnWPy/to+JKodY/aRWunfsZ1z+HUJve6pPXP4aea6UYD9g/av8e8oSL2D80c7XELwnZP+D5Lh0ZiNk/b5OL+0AI2j/hP8tfp4naPzj/7UlMDNs/b9HzuS+Q2z+NttyvURXcP4+uqCuym9w/dLlXLVEj3T871+m0LqzdP+YHX8JKNt4/dUu3VaXB3j/pofJuPk7fPz4LEQ4W3N8/u0OJGZY14D9Mi/tuwH3gP0tcXweKxuA/vra04vIP4T+hmvsA+1nhP/kHNGKipOE/wP5

In [14]:
# Set up callbacks to live update the plots
def update_data(size, Emax):
    # Generate the new curve
    k0 = np.pi/(size);
    N = int(np.sqrt((Emax*1.602e-19*2*m)/hbar**2)/(k0*1e9)); # calculate the number of allowed states by the energy constraint
    k = np.linspace(-N*k0, N*k0, 2*N);
    En = 6.242e+18*(hbar*k*1e9)**2/(2*m);
    
    update = {'xVal': k, 'yVal': En};  #Energy for a free electron in eV
    source.data = update;
    push_notebook(freeE);

    bWidth = np.amax(En)/histoN; # Define the bin width of the histogram
    kCount = np.empty(histoN, dtype = np.int8); # Declare an array to store histogram information
    Ens = range(histoN)*bWidth;
    zero = np.zeros(histoN)
    for idx in range(histoN):
        kCount[idx] = len(En[ (En > idx*bWidth) & (En <= (idx+1)*bWidth) ]);  
    update1 = {'bottom': Ens, 'top': Ens + bWidth, 'right': kCount, 'left': zero};
    source1.data = update1;
    push_notebook(histo)

    gE = 1/(np.pi*hbar)*np.sqrt(m/(2*1.602e-19*En))
    update2 = {'xVal': En, 'yVal': gE}
    source2.data = update2;
    push_notebook(gEplot);

In [15]:
show(row(freeE,histo,gEplot), notebook_handle = True);

In [16]:
interact(update_data, size = (5, 300, 10), Emax = (0.5, 2, 0.1));

interactive(children=(IntSlider(value=145, description='size', max=300, min=5, step=10), FloatSlider(value=1.2…

In [ ]:
len(En)